## Training Model

### Loading packages

In [90]:
import numpy as np
import pandas as pd
import pdb

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

import tensorflow as tf

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.layers.merge import Concatenate
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

seed = 123

### Importing data

In [2]:
expression_levels_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/expression_levels_subset.csv')
gene_variants_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/gene_variants.csv')
outcomes_raw = pd.read_csv('/Users/billyf/deepiv-gwas/derived_data/outcomes.csv')

In [3]:
# sanity check
assert(gene_variants_raw.shape[1] - 1 == outcomes_raw.shape[0])
print("Dim(z) = ", gene_variants_raw.shape)
print("Dim(p) = ", expression_levels_raw.shape)
print("Dim(y) = ", outcomes_raw.shape)

Dim(z) =  (13980, 12667)
Dim(p) =  (2344, 12667)
Dim(y) =  (12666, 2)


In [4]:
expression_levels = expression_levels_raw.drop('mrna', axis = 1).transpose().as_matrix()
gene_variants = gene_variants_raw.drop('hugo', axis = 1).transpose().as_matrix()
outcomes = outcomes_raw['outcome'].as_matrix()

In [5]:
print("Dim(z) = ", gene_variants.shape)
print("Dim(p) = ", expression_levels.shape)
print("Dim(y) = ", outcomes.shape)

Dim(z) =  (12666, 13980)
Dim(p) =  (12666, 2344)
Dim(y) =  (12666,)


In [220]:
z_train, z_test, p_train, p_test, y_train, y_test = train_test_split(gene_variants, expression_levels, outcomes, test_size=0.1, random_state=seed)

### Training first stage model

In [6]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

instruments = Input(shape=(gene_variants.shape[1],), name = "instruments")
hidden = [7000, 4000, 3000]


activation = "relu"

n_components = expression_levels.shape[1]

est_treat = architectures.feed_forward_net(instruments,
                                          lambda x: densities.mixture_of_gaussian_output(x, n_components),
                                          hidden_layers = hidden,
                                          activations = activation)

treatment_model = Treatment(inputs=[instruments], outputs=est_treat)
print("Fitting treatment")
treatment_model.compile('adam',
                       loss='mixture_of_gaussians',
                       n_components = n_components)

treatment_model.fit(gene_variants, expression_levels,
                   epochs = epochs,
                   batch_size = batch_size)

Fitting treatment
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/118


ValueError: Cannot feed value of shape (100, 2344) for Tensor 'concatenate_1_target:0', which has shape '(?, 1)'

In [199]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

instruments = Input(shape=(gene_variants.shape[1],), name = "instruments")
hidden = [200, 200, 200, 200, 200]


activation = "relu"
l2_reg = 0.0001
w_reg = regularizers.l2(l2_reg)

n_components = expression_levels.shape[1]

firstStage = Sequential([
    Dense(hidden[0], activation='relu', input_dim = 13980, name='fc1'),
    Dense(hidden[1], activation='relu', name = 'fc2'),
    Dense(hidden[2], activation='relu', name = 'fc3'),
    Dense(hidden[3], activation='relu', name = 'fc4'),
    Dense(hidden[4], activation='relu', name = 'fc5'),
    Dense(2344, activation = 'relu', name = 'output')])

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon = 1e-08)
    
firstStage.compile(optimizer='adam', loss='mse')


firstStage.fit(z_train, p_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/118
11399/11399 [==============================] - 12s - loss: 1339117.8654    
Epoch 2/118
11399/11399 [==============================] - 11s - loss: 1291711.7836    
Epoch 3/118
11399/11399 [==============================] - 11s - loss: 1185221.5010    
Epoch 4/118
11399/11399 [==============================] - 12s - loss: 741653.2536    
Epoch 5/118
11399/11399 [==============================] - 12s - loss: 532855.0658    
Epoch 6/118
11399/11399 [==============================] - 12s - loss: 433080.4761    
Epoch 7/118
11399/11399 [==============================] - 11s - loss: 373094.1020    
Epoch 8/118
11399/11399 [==============================] - 11s - loss: 352683.6638    
Epoch 9/118
11399/11399 [==============================] - 12s - loss: 340079.8035    
Epoch 10/118
11399/11399 [==============================] - 12s - loss: 329271.9627    
Epoch 11/118
11399/11399 [==============================] - 13s - loss: 322825.1349    
Epoch 12/118
11399/11399 [============

11399/11399 [==============================] - 12s - loss: 244794.7711    
Epoch 95/118
11399/11399 [==============================] - 10s - loss: 243514.6421    
Epoch 96/118
11399/11399 [==============================] - 10s - loss: 243682.7021    
Epoch 97/118
11399/11399 [==============================] - 11s - loss: 245652.1927    
Epoch 98/118
11399/11399 [==============================] - 12s - loss: 244905.2011    
Epoch 99/118
11399/11399 [==============================] - 12s - loss: 243624.7191    
Epoch 100/118
11399/11399 [==============================] - 12s - loss: 242562.8387    
Epoch 101/118
11399/11399 [==============================] - 12s - loss: 242312.4937    
Epoch 102/118
11399/11399 [==============================] - 11s - loss: 243826.1177    
Epoch 103/118
11399/11399 [==============================] - 11s - loss: 243601.1154    
Epoch 104/118
11399/11399 [==============================] - 11s - loss: 243365.3466    
Epoch 105/118
11399/11399 [=============

In [200]:
p_hat_test = firstStage.predict(z_test)

p_residuals = p_test - p_hat_test
p_mse = p_residuals * p_residuals
print("First stage of DeepIV MSE is", p_mse)

First stage of DeepIV MSE is [[1.06708900e+02 1.50327415e+01 8.89136827e+03 ... 1.26000400e+03
  3.08396764e+06 5.16652900e+00]
 [1.14490000e+02 2.49472519e+03 1.41933623e+04 ... 1.20905487e+03
  5.54086210e+04 2.58888100e+02]
 [8.31168900e+02 1.26025000e+03 3.61781587e+03 ... 1.19562595e+03
  1.34497661e+03 3.46096890e+01]
 ...
 [1.43041600e+02 2.03040360e+03 9.09009982e+04 ... 8.80031610e+03
  1.73722240e+05 8.87444100e+02]
 [1.28595600e+02 2.41797878e+02 2.17133832e+04 ... 2.84728960e+03
  4.54513644e+05 1.55500900e+02]
 [3.03050250e+01 1.34794930e+03 3.32006735e+01 ... 6.42115600e+02
  4.24360000e+04 1.05560010e+03]]


In [221]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

mrna = Input(shape=(expression_levels.shape[1],), name = "mrna")
hidden = [100, 50]


activation = "relu"
l2_reg = 0.0001
w_reg = regularizers.l2(l2_reg)

n_components = expression_levels.shape[1]

p_hat_train = firstStage.predict(z_train)


secondStage = Sequential([
    Dense(hidden[0], activation='tanh', input_dim = 2344, name='fc1'),
    Dense(hidden[1], activation = 'tanh', name = 'fc2'),
    Dense(1, activation = 'sigmoid', name = 'output')])
    
secondStage.compile(optimizer='adam', loss='binary_crossentropy')





In [222]:
secondStage.fit(p_hat_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/118
11399/11399 [==============================] - 1s - loss: 0.0796     
Epoch 2/118
11399/11399 [==============================] - 1s - loss: 0.0314     
Epoch 3/118
11399/11399 [==============================] - 0s - loss: 0.0306     
Epoch 4/118
11399/11399 [==============================] - 0s - loss: 0.0255     
Epoch 5/118
11399/11399 [==============================] - 1s - loss: 0.0240     
Epoch 6/118
11399/11399 [==============================] - 1s - loss: 0.0274     
Epoch 7/118
11399/11399 [==============================] - 1s - loss: 0.0282     
Epoch 8/118
11399/11399 [==============================] - 1s - loss: 0.0269     
Epoch 9/118
11399/11399 [==============================] - 1s - loss: 0.0263     
Epoch 10/118
11399/11399 [==============================] - 1s - loss: 0.0294     
Epoch 11/118
11399/11399 [==============================] - 1s - loss: 0.0294     
Epoch 12/118
11399/11399 [==============================] - 1s - loss: 0.0432     
Epoch 13/118


11399/11399 [==============================] - 0s - loss: 0.0257     
Epoch 100/118
11399/11399 [==============================] - 1s - loss: 0.0252     
Epoch 101/118
11399/11399 [==============================] - 1s - loss: 0.0266     
Epoch 102/118
11399/11399 [==============================] - 1s - loss: 0.0242     
Epoch 103/118
11399/11399 [==============================] - 0s - loss: 0.0278     
Epoch 104/118
11399/11399 [==============================] - 0s - loss: 0.0263     
Epoch 105/118
11399/11399 [==============================] - 0s - loss: 0.0236     
Epoch 106/118
11399/11399 [==============================] - 0s - loss: 0.0243     
Epoch 107/118
11399/11399 [==============================] - 0s - loss: 0.0249     
Epoch 108/118
11399/11399 [==============================] - 0s - loss: 0.0231     
Epoch 109/118
11399/11399 [==============================] - 0s - loss: 0.0244     
Epoch 110/118
11399/11399 [==============================] - 0s - loss: 0.0267     
Epoch 

In [ ]:


oneStage = Sequential([
    Dense(hidden[0], activation='tanh', input_dim = 2344, name='fc1'),
    Dense(hidden[1], activation = 'tanh', name = 'fc2'),
    Dense(1, activation = 'sigmoid', name = 'output')])
    
oneStage.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
oneStage.fit(p_train, y_train, epochs = epochs, batch_size = batch_size, verbose = 1)

Epoch 1/118
11399/11399 [==============================] - 1s - loss: 0.0679     
Epoch 2/118
11399/11399 [==============================] - 0s - loss: 0.0059     
Epoch 3/118
11399/11399 [==============================] - 0s - loss: 0.0033     
Epoch 4/118
11399/11399 [==============================] - 1s - loss: 0.0027     
Epoch 5/118
11399/11399 [==============================] - 1s - loss: 0.0022     
Epoch 6/118
11399/11399 [==============================] - 1s - loss: 0.0016     
Epoch 7/118
11399/11399 [==============================] - 1s - loss: 0.0014     
Epoch 8/118
11399/11399 [==============================] - 0s - loss: 0.0020     
Epoch 9/118
11399/11399 [==============================] - 0s - loss: 0.0011     
Epoch 10/118
11399/11399 [==============================] - 0s - loss: 0.0013     
Epoch 11/118
11399/11399 [==============================] - 0s - loss: 0.0011     
Epoch 12/118
11399/11399 [==============================] - 0s - loss: 0.0010       
Epoch 13/11

In [208]:
#Test

p_hat_test = firstStage.predict(z_test)
y_hat_test_deepiv = secondStage.predict(p_hat_test)
y_hat_test_naive = oneStage.predict(p_test)

In [209]:
y_hat_test_deepiv = (y_hat_test_deepiv > .5).astype(int)
y_hat_test_naive = (y_hat_test_naive > .5).astype(int)
y_hat_test_naive = y_hat_test_naive.astype(int)

In [210]:
print(y_test.shape)
print(y_hat_test_deepiv.shape)
print(y_hat_test_naive.shape)

(1267, 1)
(1267, 1)
(1267, 1)


In [217]:
y_test = y_test[:,np.newaxis]
y_test.shape

(1267, 1)

In [218]:
class_error_vec_deepiv = (y_test != y_hat_test_deepiv).astype(int)
class_error_vec_naive= (y_test != y_hat_test_naive).astype(int)

In [219]:
class_error_deepiv = np.mean(class_error_vec_deepiv)
class_error_naive = np.mean(class_error_vec_naive)
print('DeepIV classification error is', class_error_deepiv)
print('OneStage classification error is', class_error_naive)

DeepIV classification error is 0.03314917127071823
OneStage classification error is 0.0
